In [ ]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas56semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet") #Costos unitarios
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet") #ventas perdidas
vp_reales = pd.read_excel("/home/donsson/proyectos/INDICADOR NS/vp_agosto.xlsx") #vp reales


# EDA

## Facturas

In [ ]:
import re
import unicodedata

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "SUCURSAL CALI",
    "FMED": "SUCURSAL MEDELLIN",
    "FMDE":"SUCURSAL MEDELLIN",
    "FCTG": "SUCURSAL CARTAGENA",
    "FBAQ": "SUCURSAL BARRANQUILLA",
    "FVAL": "SUCURSAL VALLADOLID",
    "FCOT":"PRINCIPAL COTA",
    "FBUC":"SUCURSAL BUCARAMANGA",
    "FNOR":"SUCURSAL NORTE",
    "FCL6":"SUCURSAL CALLE 6",
    "PV2E":"SUCURSAL CALLE 6",
    "PV3E":"SUCURSAL VALLADOLID",
    "CLL6":"SUCURSAL CALLE 6",
    "PV1E":"SUCURSAL COTA" ,#Las que comienzan por p son los mostradores
    "PV4E":"SUCURSAL NORTE",
    "PV9E":"SUCURSAL CALI"

}


# Equivalencias para normalizar nombres truncados o mal escritos
mapa_equivalencias = {
    "MEDELLIN": "SUCURSAL MEDELLIN",
    "MEDELLI": "SUCURSAL MEDELLIN",
    "MEDELL": "SUCURSAL MEDELLIN",
    "MEDELI": "SUCURSAL MEDELLIN",
    "CALI": "SUCURSAL CALI",
    "CLL6":"SUCURSAL CALLE 6",
    "BUCARAMANGA":"SUCURSAL BUCARAMANGA",
    "BARRANQUILLA": "SUCURSAL BARRANQUILLA",
    "VALLADOLID": "SUCURSAL VALLADOLID",
    "CALLE 6":"SUCURSAL CALLE 6",
    "COTA":"PRINCIPAL COTA",
    "NORTE":"SUCURSAL NORTE"
}

def normalizar(texto):
    """Quita tildes y pasa a mayúsculas"""
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto.upper()

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    sucursal = None
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip()
        else:
            # 3) Buscar prefijo de código
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # Normalizar texto
    sucursal = normalizar(sucursal)
    
    # Limpiar T1, T2, T3 al final
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    # Aplicar equivalencias
    sucursal = mapa_equivalencias.get(sucursal, sucursal)
    
    return sucursal

# Aplicar al dataframe
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

## Ventas perdidas

In [ ]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)["ventas_perdidas"]
    .sum()
    .rename(columns={"ventas_perdidas": "lost_sales"})
)

# Mostrar resultado agrupado
vp_week = lost_by_week


In [ ]:
vp_reales["product_ref"] = vp_reales["Descripcion"].str.extract(r"\[([A-Z0-9]+)\]")
vp_reales.head()

# Asegurar que ambos son strings para evitar problemas
vp_week["product_ref"] = vp_week["product_ref"].astype(str)
vp_reales["product_ref"] = vp_reales["product_ref"].astype(str)

# 1. Obtener listas únicas
refs_week = set(vp_week["product_ref"].unique())
refs_real = set(vp_reales["product_ref"].unique())

# 2. Diferencia: los que están en vp_week pero no en vp_real
refs_extra = refs_week - refs_real

# 3. Filtrar el dataframe para verlos completos
df_discrepantes = vp_week[vp_week["product_ref"].isin(refs_extra)]


df_discrepantes = df_discrepantes[(df_discrepantes["mes"]==8) & (df_discrepantes["lost_sales"]>0) ]
df_discrepantes = df_discrepantes.groupby("product_ref").agg({"lost_sales":"sum"})
print("Cantidad de vp que no deberia tomar:", df_discrepantes["lost_sales"].sum())
df_discrepantes #Los productos que no se movieron hace mucho tiempo no salen en el analisis de ns

In [ ]:
vp_agosot_2025 = vp_week[(vp_week["ano"]==2025) & (vp_week["mes"]==8) ]
vp_agosot_2025.to_excel("vp_revisar.xlsx")

vp_agosot_2025.query("product_ref == 'DAB28118025' and store_name == 'PRINCIPAL COTA'")
#vp_agosot_2025.groupby("store_name")["lost_sales"].sum()

# UNION

## EMA SEMANAL CON VP SEMANALES (SOLO 2025)

In [ ]:
df.head()

In [ ]:
# ===============================
# Procesar ventas normales
# ===============================
df_sales = df.copy()
df_sales["date_invoice"] = pd.to_datetime(df_sales["date_invoice"], errors="coerce")


# Referncia de producto
df_sales["product_ref"] = df_sales["product_name"].str.extract(r"\[([A-Z0-9]+)\]")


# Columnas temporales igual que en df_vp
df_sales["Semana"] = df_sales["date_invoice"].dt.to_period("W").dt.start_time
df_sales["ano"]    = df_sales["Semana"].dt.year
df_sales["mes"]    = df_sales["Semana"].dt.month
df_sales["dia"]    = df_sales["Semana"].dt.day

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
sales_by_week = (
    df_sales.groupby(["Sucursal", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)["quantity"]
    .sum()
    .rename(columns={"quantity": "sales",
                     "Sucursal":"store_name"})
)

# Resultado
sales_by_week.sample(10)


In [ ]:
df_merged = pd.merge(
    sales_by_week[["store_name", "product_ref", "Semana", "sales"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)


In [ ]:
df_merged["año"]    = df_merged["Semana"].dt.year
df_merged["mes"]    = df_merged["Semana"].dt.month
df_merged["dia"]    = df_merged["Semana"].dt.day

df_merged.head(10)

## NORMALIZAR DF DE COSTOS

In [ ]:
df_p["product_ref"] = df_p["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df_p_unique = (
    df_p[["product_ref", "producto_costo_unitario"]]
    .drop_duplicates(subset=["product_ref"])
)



df_p.head()

### UNIR COSTO

In [ ]:
df_merge_def = pd.merge(
    df_merged,
    df_p_unique,
    on="product_ref",
    how="left"
).fillna(0)


merge_def = df_merge_def[df_merge_def["producto_costo_unitario"] !=0].copy() #Eliminar productos sin costos unitarios

merge_def.head(10)

In [ ]:
import pandas as pd
import numpy as np

def compute_demand_and_ema(df,
                           alpha=0.20,        # 20% como en tu config
                           n_init_weeks=12,   # semanas que usa el proceso (rango de evaluacion)
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales"):
    df = df.copy()

    # ---------- Asegurar tipos y semana iniciando lunes ----------
    # Si Semana no es datetime, intentamos convertir
    df[week_col] = pd.to_datetime(df[week_col], errors="coerce")

    # Normalizar semanas al lunes inicio (start of week, lunes)
    # Esto genera el timestamp del lunes de la semana ISO correspondiente
    # (equivalente al comportamiento del código original)
    df[week_col] = df[week_col].dt.to_period('W-MON').dt.start_time

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # Orden y agrupación
    df = df.sort_values(["store_name", "product_ref", week_col])

    out_groups = []

    # Recorremos por tienda+producto
    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        # Inicial EMA (EMA_0) -> si hay suficientes semanas, usamos el promedio de las primeras n_init_weeks sales
        # Si hay menos semanas, usamos media de las sales disponibles.
        if L == 0:
            out_groups.append(g)
            continue

        # inicialización: usar promedio de 'sales' de las primeras n_init_weeks (o de lo que haya)
        init_n = min(n_init_weeks, L)
        # si no hay sales (todos ceros), ema_prev será 0
        ema_prev = float(np.nanmean(sales[:init_n])) if init_n > 0 else 0.0
        if np.isnan(ema_prev):
            ema_prev = 0.0

        # Iterar semanas y aplicar reglas del documento
        for i in range(L):
            s = sales[i]
            l = lost[i]

            # Regla 1: si ventas >= 2 * ventas_perdidas
            if s >= 2.0 * l:
                demand_candidate = s + l
                # aplicar tope MAX = 1.5 * ventas
                # (si s == 0 ese caso no entra porque s >= 2*l sería falso cuando l>0)
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                # Regla 2: ventas < 2 * ventas_perdidas
                # demanda = ventas + 0.5 * EMA(t-1)
                demand = s + 0.5 * ema_prev

            # Guardar demanda
            demanda[i] = demand

            # Calcular EMA (iterativo) con alpha
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema =round(ema,2)
            ema_arr[i] = ema

            # actualizar para la próxima semana
            ema_prev = ema

        # Añadir columnas al grupo
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr

        out_groups.append(g)

    # Concat y devolver
    result = pd.concat(out_groups, ignore_index=True, sort=False)

    # Mantener mismo orden original
    result = result.sort_values(["store_name", "product_ref", week_col]).reset_index(drop=True)
    return result

# ------------------ USO ------------------
# suponiendo merge_def es tu df final
# ajusta alpha y n_init_weeks si quieres (alpha=0.2, n_init_weeks=12 por defecto)
df_with_demand = compute_demand_and_ema(merge_def, alpha=0.20, n_init_weeks=12)

# ver primeras filas
df_with_demand[["store_name","product_ref","Semana","sales","lost_sales","demanda_ajustada","EMA"]].head(20)


In [ ]:
df_with_demand["semana_num"] = df_with_demand["Semana"].dt.isocalendar().week

In [ ]:


df_demand_2025 = df_with_demand[df_with_demand["año"]==2025]


demand_2025 = df_demand_2025[["store_name","product_ref","año","semana_num","EMA","producto_costo_unitario","demanda_ajustada"]]

filtro_bq = demand_2025[(demand_2025["store_name"]=="SUCURSAL BARRANQUILLA") & (demand_2025["semana_num"]==36)].sort_values(by=("EMA"),ascending=False)

filtro_bq.head(20)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ---------- Funciones auxiliares ----------

def get_first_day_of_week(year: int) -> datetime:
    """
    Reproduce la lógica del Odoo original:
    - Si el 1 de enero cae jueves o después → semana 1 empieza en el siguiente lunes
    - Si cae lunes, martes o miércoles → retrocede al lunes anterior
    """
    jan1 = datetime(year, 1, 1)
    dow = jan1.weekday()  # lunes=0, domingo=6
    if dow >= 3:  # jueves (3), viernes (4), sábado (5), domingo (6)
        # semana 1 arranca el lunes siguiente
        return jan1 + timedelta(days=(7 - dow))
    else:
        # semana 1 arranca el lunes anterior o el mismo día si ya es lunes
        return jan1 - timedelta(days=dow)


def get_start_end_week(year: int, week_num: int):
    """Devuelve lunes y domingo de la semana solicitada, siguiendo get_first_day_of_week"""
    first_monday = get_first_day_of_week(year)
    start_date = first_monday + timedelta(weeks=week_num - 1)
    end_date = start_date + timedelta(days=6)
    return start_date, end_date


# ---------- Función principal ----------

def compute_demand_and_ema(df,
                           alpha=0.20,
                           n_init_weeks=12,
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales"):

    df = df.copy()

    # Normalizar semanas con la misma lógica Odoo (get_start_end_week)
    df[week_col] = pd.to_datetime(df[week_col], errors="coerce")
    df["year"] = df[week_col].dt.year
    df["week"] = df[week_col].dt.isocalendar().week

    # Reemplazar la semana por el lunes base según Odoo
    df[week_col] = df.apply(
        lambda r: get_start_end_week(int(r["year"]), int(r["week"]))[0],
        axis=1
    )

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # Orden
    df = df.sort_values(["store_name", "product_ref", week_col])

    out_groups = []

    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        if L == 0:
            out_groups.append(g)
            continue

        # --- Paso 1: calcular demandas de las primeras n_init_weeks
        init_n = min(n_init_weeks, L)
        demanda_inicial = []

        ema_prev = 0.0
        for i in range(init_n):
            s, l = sales[i], lost[i]

            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                demand = s + 0.5 * ema_prev  # aquí ema_prev todavía es el inicial 0 para i=0

            demanda[i] = demand
            demanda_inicial.append(demand)
            ema_prev = demand  # solo para el seed (no aplica alpha aquí)

        # EMA inicial: promedio de demandas ajustadas
        ema_prev = np.mean(demanda_inicial) if len(demanda_inicial) > 0 else 0.0
        ema_prev = round(ema_prev, 2)

        # Guardamos EMA inicial
        ema_arr[:init_n] = ema_prev

        # --- Paso 2: resto de semanas con recursión
        for i in range(init_n, L):
            s, l = sales[i], lost[i]

            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                demand = s + 0.5 * ema_prev

            demanda[i] = demand
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema = round(ema, 2)  # redondear cada semana
            ema_arr[i] = ema
            ema_prev = ema

        # Añadir columnas
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr
        out_groups.append(g)

    result = pd.concat(out_groups, ignore_index=True, sort=False)
    result = result.sort_values(["store_name", "product_ref", week_col]).reset_index(drop=True)
    return result


In [ ]:
definitivo = compute_demand_and_ema(merge_def,
                           alpha=0.20,
                           n_init_weeks=12,
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales")


In [ ]:
definitivo.head()

In [ ]:
demand_2025_2 = definitivo[definitivo["año"]==2025]

In [ ]:
demand_2025_2["semana_num"] = demand_2025_2["Semana"].dt.isocalendar().week

demand_2025_2 = demand_2025_2[["store_name","product_ref","año","semana_num","EMA","producto_costo_unitario","demanda_ajustada"]]

filtro_bq_2 = demand_2025_2[(demand_2025_2["store_name"]=="SUCURSAL BARRANQUILLA") & (demand_2025_2["semana_num"]==36)].sort_values(by=("EMA"),ascending=False)

filtro_bq_2.head(20)